In [0]:
%pip install tabulate
%restart_python


In [0]:
from databricks.sdk import WorkspaceClient
from tabulate import tabulate
from typing import Any, Dict, List, Optional
import time
import pprint

In [0]:
space_id = dbutils.widgets.get("genie-space-id")
print(space_id)


In [0]:
w = WorkspaceClient()

In [0]:
print(w.genie.get_space(space_id=space_id))

In [0]:
# Start a new conversation — ask a question in natural language
question = "What are the top 10 suppliers in AMER?"


In [0]:
def extract_genie_response(w: Any, message: Any) -> Dict[str, Any]:
    """
    Extracts all useful content from a GenieMessage:
    - Text attachments (natural language responses)
    - Query attachments (SQL, description, metadata, and full results if available)
    - Fallbacks to message.query_result or message.content if needed

    Returns a dict shaped like:
    {
        "text": [str, ...],
        "queries": [
            {
                "sql": str | None,
                "description": str | None,
                "metadata": Any | None,
                "attachment_id": str,
                "columns": [str, ...],
                "rows": [ [Any, ...], ... ],
                "raw": Any,   # full_result object
                "error": str | None,
            },
            ...
        ],
        "query_result": str | None,  # fallback if no text attachments
        "prompt": str | None         # fallback (usually the prompt you sent)
    }
    """
    result: Dict[str, Any] = {
        "text": [],
        "queries": [],
        "query_result": None,
        "prompt": None,
    }

    attachments = getattr(message, "attachments", []) or []

    # 1) Collect text attachment responses
    for att in attachments:
        text_obj = getattr(att, "text", None)
        if text_obj and hasattr(text_obj, "content") and text_obj.content:
            result["text"].append(text_obj.content)

    # 2) Collect SQL/query attachments, fetch full result
    for att in attachments:
        query_obj = getattr(att, "query", None)
        if not query_obj:
            continue

        q_info: Dict[str, Any] = {
            "sql": getattr(query_obj, "query", None),
            "description": getattr(query_obj, "description", None),
            "metadata": getattr(query_obj, "query_result_metadata", None),
            "attachment_id": getattr(att, "attachment_id", None),
            "columns": [],
            "rows": [],
            "raw": None,
            "error": None,
        }

        # Fetch the full query result if possible
        try:
            full_result = w.genie.get_message_attachment_query_result(
                space_id=getattr(message, "space_id", None),
                conversation_id=getattr(message, "conversation_id", None),
                message_id=getattr(message, "id", None),
                attachment_id=getattr(att, "attachment_id", None),
            )
            q_info["raw"] = full_result

            # Extract columns and rows safely
            sr = getattr(full_result, "statement_response", None)

            # Columns
            columns: List[str] = []
            try:
                manifest = getattr(sr, "manifest", None)
                schema = getattr(manifest, "schema", None) if manifest else None
                cols = getattr(schema, "columns", None) if schema else None
                if cols:
                    columns = [getattr(col, "name", "") for col in cols]
            except Exception:
                # Leave columns empty if structure doesn't match
                pass

            # Rows
            rows: List[List[Any]] = []
            try:
                result_obj = getattr(sr, "result", None)
                data_array = getattr(result_obj, "data_array", None) if result_obj else None
                if data_array is not None:
                    rows = data_array
            except Exception:
                # Leave rows empty if structure doesn't match
                pass

            q_info["columns"] = columns
            q_info["rows"] = rows

        except Exception as e:
            q_info["error"] = str(e)

        result["queries"].append(q_info)

    # 3) Fallbacks if no text responses were found
    if not result["text"] and getattr(message, "query_result", None):
        result["query_result"] = str(getattr(message, "query_result"))

    if not result["text"] and getattr(message, "content", None):
        result["prompt"] = getattr(message, "content")

    return result



In [0]:
def print_genie_response(bundle: Dict[str, Any], mode: str = "pretty") -> None:
    """
    Print the extracted response bundle with a toggle:
      - mode="pretty": readable tables (uses 'tabulate' if available)
      - mode="raw": raw lists/objects + full attachment dump where available

    Expected bundle shape (from `extract_genie_response`):
    {
        "text": [str, ...],
        "queries": [
            {
                "sql": str | None,
                "description": str | None,
                "metadata": Any | None,
                "attachment_id": str,
                "columns": [str, ...],
                "rows": [ [Any, ...], ... ],
                "raw": Any,
                "error": str | None,
            },
            ...
        ],
        "query_result": str | None,
        "prompt": str | None
    }
    """
    if mode not in ("pretty", "raw"):
        raise ValueError("mode must be 'pretty' or 'raw'")

    pp = pprint.PrettyPrinter(indent=2, width=100, compact=False)

    # Text responses
    texts = bundle.get("text") or []
    if texts:
        hdr = "=== Text Responses ===" if mode == "pretty" else "=== Text Responses (raw) ==="
        print(hdr)
        for i, t in enumerate(texts, 1):
            print(f"{i}. {t}")

    # Query responses
    queries = bundle.get("queries") or []
    if queries:
        hdr = "\n=== Query Attachments ===" if mode == "pretty" else "\n=== Query Attachments (raw) ==="
        print(hdr)
        for i, q in enumerate(queries, 1):
            print(f"\n[{i}] Attachment ID: {q.get('attachment_id')}")
            if q.get("sql"):
                print("SQL:", q["sql"])
            if q.get("description"):
                print("Description:", q["description"])

            metadata = q.get("metadata")
            if metadata is not None:
                if mode == "pretty":
                    print("Metadata:", metadata)
                else:
                    print("Metadata (raw):")
                    pp.pprint(metadata)

            if q.get("error"):
                print("Error retrieving query result:", q["error"])

            cols = q.get("columns") or []
            rows = q.get("rows") or []

            if cols and rows:
                if mode == "pretty":
                    print("\n=== Full Query Result ===")
                    if tabulate:
                        print(tabulate(rows, headers=cols, tablefmt="fancy_grid"))
                    else:
                        print("Columns:", cols)
                        print("Rows:")
                        for r in rows:
                            print(r)
                else:
                    print("\nColumns (raw):")
                    pp.pprint(cols)
                    print("\nRows (raw):")
                    pp.pprint(rows)

            # Dump the full raw attachment object when in raw mode
            if mode == "raw":
                raw_obj = q.get("raw")
                if raw_obj is not None:
                    print("\nFull Attachment Response (raw):")
                    dumped = None
                    try:
                        if hasattr(raw_obj, "to_dict") and callable(getattr(raw_obj, "to_dict")):
                            dumped = raw_obj.to_dict()
                        elif hasattr(raw_obj, "model_dump") and callable(getattr(raw_obj, "model_dump")):
                            dumped = raw_obj.model_dump()
                        elif hasattr(raw_obj, "__dict__"):
                            dumped = raw_obj.__dict__
                    except Exception:
                        dumped = None

                    if dumped is not None:
                        pp.pprint(dumped)
                    else:
                        print(repr(raw_obj))

    # Fallbacks
    if bundle.get("query_result"):
        if mode == "pretty":
            print("\n=== query_result Fallback ===")
            print(bundle["query_result"])
        else:
            print("\n=== query_result Fallback (raw) ===")
            pp.pprint(bundle["query_result"])

    if bundle.get("prompt"):
        if mode == "pretty":
            print("\n=== Prompt Fallback ===")
            print(bundle["prompt"])
        else:
            print("\n=== Prompt Fallback (raw) ===")
            pp.pprint(bundle["prompt"])

### Start Conversation and Wait - Blocking

In [0]:
# This method sends the request and waits until the reply (or query) is ready
message_1 = w.genie.start_conversation_and_wait(
    space_id = space_id,
    content = question
)

# The returned object is a GenieMessage (per current SDK behavior) — it contains message metadata and attachments
#print("Message metadata:", message_1)


In [0]:
follow_up_question = "Give me the summary of the above finding with detailed analysis."

In [0]:
message_2 = w.genie.create_message_and_wait(
    space_id = space_id,
    conversation_id = message_1.conversation_id,
    content = follow_up_question
)
#print("Message metadata:", message_2)

In [0]:
bundle = extract_genie_response(w, message_2)
print_genie_response(bundle, mode="raw")


In [0]:
bundle = extract_genie_response(w, message_1)
print_genie_response(bundle, mode="pretty")

### Start Conversation - Non Blocking

In [0]:
# This immediately returns the initial message
message = w.genie.start_conversation(
    space_id=space_id,
    content=question
)

conversation_id = message.conversation_id
message_id = message.message_id
response = message.response
print("Conversation ID:", conversation_id)
print("Message ID:", message_id)
print("Message response:", response)

In [0]:
POLL_INTERVAL = 2
MAX_WAIT = 60
reply_message = None
start_time = time.time()

while time.time() - start_time < MAX_WAIT:
    msg = w.genie.get_message(
        space_id=space_id,
        conversation_id=conversation_id,
        message_id=message_id
    )
    #print(f"----------\n{msg}\n-----------\n")
    # Check message status
    if msg.status.name == "COMPLETED":
        reply_message = msg
        break
    elif msg.status.name == "FAILED":
        raise RuntimeError(f"Genie failed: {msg.status.error}")
    
    time.sleep(POLL_INTERVAL)

if not reply_message:
    raise TimeoutError("Timed out waiting for Genie reply.")

print("\n=== Genie Message Metadata ===")
print(reply_message)




In [0]:
if reply_message.attachments:
    for att in reply_message.attachments:
        if att.query:
            print("\n=== Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            if att.query.query_result_metadata:
                print("\n=== Result Metadata ===")
                print(att.query.query_result_metadata)

            # -------------------------------
            # Deprecated: fetch the full query result
            # -------------------------------
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=reply_message.id,
                attachment_id=att.attachment_id
            )

            # Extract column names and rows from the full_result object
            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array

            # Pretty-print the query result
            print("\n=== Full Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("\nNo attachments in response.")

### Follow Ups - With create_message() - Non-Blocking

In [0]:
follow_up_question='What are the top 10 suppliers in the UNITED STATES by account balance?'

In [0]:
waiter = w.genie.create_message(
    space_id=space_id,
    conversation_id=conversation_id,
    content=follow_up_question
)

In [0]:
POLL_INTERVAL = 2
MAX_WAIT = 60
follow_reply = None
start_time = time.time()

while time.time() - start_time < MAX_WAIT:
    msg = w.genie.get_message(
        space_id=space_id,
        conversation_id=conversation_id,
        message_id=waiter.message_id
    )

    if msg.status.name == "COMPLETED":
        follow_reply = msg
        break
    elif msg.status.name == "FAILED":
        raise RuntimeError(f"Genie failed to process the follow-up message: {msg.status.error}")

    time.sleep(POLL_INTERVAL)

if not follow_reply:
    raise TimeoutError("Timed out waiting for follow-up reply.")

print("\n=== Follow-up Message Metadata ===")
print(follow_reply)



In [0]:
# Process attachments and print full query results
if follow_reply.attachments:
    for att in follow_reply.attachments:
        if att.query:
            print("\n=== Follow-up Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            # Full query result
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=follow_reply.id,
                attachment_id=att.attachment_id
            )

            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array
            print("\n=== Full Follow-up Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("No attachments in follow-up response.")


### Follow Ups - With create_message_and_wait() - Blocking

In [0]:
follow_reply = w.genie.create_message_and_wait(
    space_id=space_id,
    conversation_id=conversation_id,
    content=follow_up_question
)

print("\n=== Follow-up Message Metadata ===")
print(follow_reply)


In [0]:
# Process attachments and print full query results
if follow_reply.attachments:
    for att in follow_reply.attachments:
        if att.query:
            print("\n=== Follow-up Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            # Full query result
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=follow_reply.id,
                attachment_id=att.attachment_id
            )

            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array
            print("\n=== Full Follow-up Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("No attachments in follow-up response.")


### With Exponential backoff

In [0]:
# This immediately returns the initial message
message = w.genie.start_conversation(
    space_id=space_id,
    content=question
)

conversation_id = message.conversation_id
message_id = message.message_id
response = message.response

print("Conversation ID:", conversation_id)
print("Message ID:", message_id)
print("Message response:", response)

In [0]:
# -------------------------------
# Polling with exponential backoff
# -------------------------------
reply_message = None
start_time = time.time()
timeout_seconds = 600   # 10 minutes max
poll_interval = 1       # initial poll interval in seconds
max_interval = 60       # max backoff interval

while time.time() - start_time < timeout_seconds:
    msg = w.genie.get_message(
        space_id=space_id,
        conversation_id=conversation_id,
        message_id=message_id
    )

    # Check status
    if msg.status.name in ["COMPLETED", "FAILED", "CANCELLED"]:
        reply_message = msg
        print(time.time() - start_time)
        break

    # Sleep and apply exponential backoff
    time.sleep(poll_interval)
    poll_interval = min(poll_interval * 2, max_interval)

if not reply_message:
    raise TimeoutError("Timed out waiting for Genie reply (10 minutes).")

# -------------------------------
# Print message metadata
# -------------------------------
print("\n=== Genie Message Metadata ===")
print(reply_message)

In [0]:
# -------------------------------
# Process attachments and full query result
# -------------------------------
if reply_message.attachments:
    for att in reply_message.attachments:
        if att.query:
            print("\n=== Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            if att.query.query_result_metadata:
                print("\n=== Result Metadata ===")
                print(att.query.query_result_metadata)

            # Deprecated: fetch the full query result
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=reply_message.id,
                attachment_id=att.attachment_id
            )

            # Extract column names and rows
            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array

            # Pretty-print the table
            print("\n=== Full Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("\nNo attachments in response.")
